In [3]:
import math
from operator import itemgetter


#读取电影数据
def load_movielens(path='./movielens/ml-100k'):
    # get movie titles
    movies = {}
    for line in open(path + '/u.item', encoding='latin-1'):
        id, title = line.split('|')[0:2]
        movies[id] = title
    # load data
    prefs = {}
    for line in open(path + '/u.data', encoding='latin-1'):
        user, movieid, rating, ts = line.split('\t')
        prefs.setdefault(user, {})
        prefs[user][movies[movieid]] = float(rating)
    return prefs, movies

# 建立物品-用户的倒序表，并计算用户相似度
def usersim(dicc):
    item_user = {}
    for u, items in dicc.items():
        for i in items.keys():
            if i not in item_user.keys():
                item_user[i] = set()
            item_user[i].add(u)
    c = {}
    n = {}
    for item, users in item_user.items():
        for u in users:
            if u not in n.keys():
                n[u] = 0
            n[u] += 1
            for v in users:
                if u == v:
                    continue
                if (u,v) not in c.keys():
                    c[u,v] = 0
                c[u,v] += 1
    w = {}
    for co_user, cuv in c.items():
        w[co_user] = cuv / math.sqrt(n[co_user[0]] * n[co_user[1]])
    return w

# 根据用户相似度，向用户推荐相似用户的产品
def match(user, dicc, w, k):
    rvi = 1
    rank = {}
    related_user= []
    interacted_items = dicc[user].keys()
    for co_user, item in w.items():
        if co_user[0] == user:
            related_user.append((co_user[1], item))
    for v, wuv in sorted(related_user, key=itemgetter(1), reverse=True)[0: k]:
        for i in dicc[v]:
            if i in dicc[v]:
                if i in interacted_items:
                    continue
                if i not in rank.keys():
                    rank[i] = 0
                rank[i] += wuv * rvi
    return rank

prefs, movies = load_movielens()
w = usersim(prefs)
last = match('176', prefs, w, 3)
print(last.keys())


dict_keys(['Great White Hype, The (1996)', "Things to Do in Denver when You're Dead (1995)", 'Georgia (1995)', 'Broken Arrow (1996)', 'Breaking the Waves (1996)', "Antonia's Line (1995)", 'Breakdown (1997)', 'Celluloid Closet, The (1995)', 'Big Night (1996)', 'Looking for Richard (1996)', 'Phenomenon (1996)', 'Tin Cup (1996)', 'Twister (1996)', 'Grosse Pointe Blank (1997)', 'Bottle Rocket (1996)', 'Toy Story (1995)', 'Emma (1996)', 'Courage Under Fire (1996)', 'Mars Attacks! (1996)', 'Wonderland (1997)', 'Leaving Las Vegas (1995)', 'Dead Man Walking (1995)', 'Get on the Bus (1996)', 'Anne Frank Remembered (1995)', 'Absolute Power (1997)', 'Restoration (1995)', 'Ransom (1996)', 'Lone Star (1996)', 'Bed of Roses (1996)', 'My Fellow Americans (1996)', 'Jackie Brown (1997)', 'Starship Troopers (1997)', 'Saint, The (1997)', 'Life Less Ordinary, A (1997)', 'Kolya (1996)', 'One Fine Day (1996)', 'Kids in the Hall: Brain Candy (1996)', 'White Squall (1996)', 'Traveller (1997)', 'Multiplicity (